In [1]:
import yahoo_fin.stock_info as si
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from pandas import read_html
#import FunctionFile as FF
from ipynb.fs.full.FunctionFile import RiskVal_func
from ipynb.fs.full.FunctionFile import stock_monte_carlo_func
from ipynb.fs.full.FunctionFile import GrahamInVal_func
from ipynb.fs.full.FunctionFile import WACC_func
from ipynb.fs.full.FunctionFile import EPSMult_func

nan=np.nan

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

from pandas_datareader import data
from datetime import datetime

In [3]:
#****START HERE*****
#****Computes Graham Intrinsic Value, calls GrahamInVal_func function from FunctionFile
#****Inputs to provide : stock_list
#****Outputs in stock_sum dataframe

stock_list = ['PYPL','AAPL','NVDA','AMD','MSFT']

stock_sum = pd.DataFrame(np.arange(7*len(stock_list)).reshape((len(stock_list),7)),
                                   columns=['Sticker',
                                            'GrahamInVal($)',
                                            'PrevClose($)',
                                            'Decision', 
                                            'Margin(%)',
                                            'EPS',
                                            'GrowthEst(%)'])

for i in stock_list:
    ami = GrahamInVal_func(i)
    ami1 = pd.DataFrame.from_dict(ami, orient='index')
    ami2 = ami1.transpose()
    stock_sum = stock_sum.append(ami2)

stock_sum.set_index('Sticker', inplace=True)
stock_sum.drop(stock_sum.index[0:len(stock_list)], inplace=True)
stock_sum

,GrahamInVal($),PrevClose($),Decision,Margin(%),EPS,GrowthEst(%)
Sticker,,,,,,
PYPL,126.66,194.57,SELL,53.62,2.18,23.37
AAPL,491.04,462.25,BUY,-5.86,13.19,12.46
NVDA,243.89,490.43,SELL,101.09,5.35,16.83
AMD,42.66,81.66,SELL,91.42,0.51,36.72
MSFT,242.42,210.98,BUY,-12.97,5.76,15


In [4]:
#****Computes Weighted Average Cost of Capital (WACC), calls WACC_func function from FunctionFile
#****Inputs to provide : stock_list
#****Outputs in stock_anal0 = concatenated WACC_df & stock_sum

WACC_df = pd.DataFrame(columns = ['WACC'], index = stock_list)

for x in stock_list:
    WACC_df['WACC'][x] = WACC_func(x)

stock_anal0=pd.concat([stock_sum, WACC_df], axis=1, sort=False)
stock_anal0

,GrahamInVal($),PrevClose($),Decision,Margin(%),EPS,GrowthEst(%),WACC
Sticker,,,,,,,
PYPL,126.66,194.57,SELL,53.62,2.18,23.37,0.0996676
AAPL,491.04,462.25,BUY,-5.86,13.19,12.46,0.0998828
NVDA,243.89,490.43,SELL,101.09,5.35,16.83,0.111806
AMD,42.66,81.66,SELL,91.42,0.51,36.72,0.185899
MSFT,242.42,210.98,BUY,-12.97,5.76,15,0.0756964


In [5]:
#****Computes Intrisic Value by EPS Multiplier Method, calls EPSMult_func function from FunctionFile
#****Inputs to provide : stock_list
#                        eps (from stock_anal0)
#                        growthest (from stock_anal0)
#                        wacc (from stock_anal0)
#****Outputs in stock_anal1 = concatenated EPSMult_df & stock_anal0

EPSMult_df = pd.DataFrame(columns = ['EPSMult_InVal'], index = stock_list)

for i in stock_list:
    EPSMult_df['EPSMult_InVal'][i] = EPSMult_func(stock_anal0['EPS'][i],stock_anal0['GrowthEst(%)'][i],stock_anal0['WACC'][i])

stock_anal1=pd.concat([stock_anal0, EPSMult_df], axis=1, sort=False)
stock_anal1

,GrahamInVal($),PrevClose($),Decision,Margin(%),EPS,GrowthEst(%),WACC,EPSMult_InVal
Sticker,,,,,,,,
PYPL,126.66,194.57,SELL,53.62,2.18,23.37,0.0996676,161.413
AAPL,491.04,462.25,BUY,-5.86,13.19,12.46,0.0998828,359.252
NVDA,243.89,490.43,SELL,101.09,5.35,16.83,0.111806,219.568
AMD,42.66,81.66,SELL,91.42,0.51,36.72,0.185899,66.1669
MSFT,242.42,210.98,BUY,-12.97,5.76,15,0.0756964,225.723


In [6]:
#****Computes Value at Risk, calls RiskVal_func function from FunctionFile
#****Inputs to provide : stock_list
#                       lookbackperiod (in years)
#****Outputs in stock_anal2 = concatenated risk_df & stock_anal1

lookbackperiod=1
risk_df = RiskVal_func(stock_list,lookbackperiod)

stock_anal2=pd.concat([stock_anal1, risk_df], axis=1, sort=False)
stock_anal2

,GrahamInVal($),PrevClose($),Decision,Margin(%),EPS,GrowthEst(%),WACC,EPSMult_InVal,1yr/95% WC Risk%,1yr/99% WC Risk%
Sticker,,,,,,,,,,
PYPL,126.66,194.57,SELL,53.62,2.18,23.37,0.0996676,161.413,-3.934097,-7.289909
AAPL,491.04,462.25,BUY,-5.86,13.19,12.46,0.0998828,359.252,-3.425843,-7.209291
NVDA,243.89,490.43,SELL,101.09,5.35,16.83,0.111806,219.568,-4.296391,-7.765302
AMD,42.66,81.66,SELL,91.42,0.51,36.72,0.185899,66.1669,-4.872821,-9.458027
MSFT,242.42,210.98,BUY,-12.97,5.76,15,0.0756964,225.723,-3.643095,-6.908890


In [7]:
#****Computes Monte Carlo Value at Risk, calls stock_monte_carlo_func function from FunctionFile
#****Inputs to provide : stock_list
#                        lookbackperiod (in years) defined above for RiskVal
#                        lookaheadperiod (in days) for stock estimate
#                        runs (for MC run size)
#****Outputs in stock_anal3 = concatenated MC_df & stock_anal2

# lookbackperiod=1 (defined already above)
lookaheadperiod=365
runs = 10000
days=lookaheadperiod

end = datetime.now()
start = datetime(end.year-lookbackperiod, end.month, end.day)
for x in stock_list:
    globals()[x] = data.DataReader(x,'yahoo',start,end)
        
closing_df = data.DataReader(stock_list,'yahoo',start,end)['Adj Close']
tech_rets = closing_df.pct_change()
rets = tech_rets.dropna()

MC_df = pd.DataFrame(columns = ['start_price','mu','sigma','1yr/99%/q','Var_WC','1Yr/99%/Risk%'], index = stock_list)

for i in stock_list:
    MC_df['mu'][i] = rets.mean()[i]
    MC_df['sigma'][i] = rets.std()[i]
    MC_df['start_price'][i] = float(closing_df.tail(1)[i])
        
    simulations = np.zeros(runs)

    for run in range(runs):
        simulations[run]=stock_monte_carlo_func(MC_df['start_price'][i],days,MC_df['mu'][i],MC_df['sigma'][i])[days-1]
    
    MC_df['1yr/99%/q'][i] = np.percentile(simulations,1)
    MC_df['Var_WC'][i] = MC_df['1yr/99%/q'][i]-MC_df['start_price'][i]
    MC_df['1Yr/99%/Risk%'][i] = 100*(MC_df['1yr/99%/q'][i]-MC_df['start_price'][i])/MC_df['start_price'][i]

stock_anal3=pd.concat([stock_anal2, MC_df], axis=1, sort=False)
stock_anal3.drop(columns=['mu','sigma','Var_WC'],inplace=True)
stock_anal3

,GrahamInVal($),PrevClose($),Decision,Margin(%),EPS,GrowthEst(%),WACC,EPSMult_InVal,1yr/95% WC Risk%,1yr/99% WC Risk%,start_price,1yr/99%/q,1Yr/99%/Risk%
Sticker,,,,,,,,,,,,,
PYPL,126.66,194.57,SELL,53.62,2.18,23.37,0.0996676,161.413,-3.934097,-7.289909,192.44,180.241,-6.33924
AAPL,491.04,462.25,BUY,-5.86,13.19,12.46,0.0998828,359.252,-3.425843,-7.209291,462.83,438.377,-5.28342
NVDA,243.89,490.43,SELL,101.09,5.35,16.83,0.111806,219.568,-4.296391,-7.765302,485.54,451.281,-7.05577
AMD,42.66,81.66,SELL,91.42,0.51,36.72,0.185899,66.1669,-4.872821,-9.458027,81.09,75.2096,-7.25175
MSFT,242.42,210.98,BUY,-12.97,5.76,15,0.0756964,225.723,-3.643095,-6.908890,209.7,198.327,-5.42325
